In [6]:
import gym
import random
import numpy as np
import time
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from gym.envs.registration import registry, register
from IPython.display import clear_output

Instructions for updating:
non-resource variables are not supported in the long term


In [7]:
try:
    register(
        id='FrozenLakeNoSlip-v0',
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name' : '4x4', 'is_slippery' : False},
        max_episode_steps=100,
        reward_threshold=0.70, # optimum = 0.74
    )
except:
    pass

env_name = 'FrozenLakeNoSlip-v0'
env = gym.make(env_name)
print(env.observation_space)
print(env.action_space)

Exception ignored in: <function QNAgent.__del__ at 0x7f010705e8b0>
Traceback (most recent call last):
  File "<ipython-input-4-ea7cabc1dd3d>", line 50, in __del__
AttributeError: 'QNAgent' object has no attribute 'sess'


Discrete(16)
Discrete(4)


In [8]:
class Agent():
    def __init__(self, env):
        self.is_discrete = type(env.action_space == gym.spaces.discrete.Discrete)
        
        if self.is_discrete:
            self.action_size = env.action_space.n
            print("action size", self.action_size)
        else:
            self.action_low = env.action_space.low
            self.action_high = env.action_space.high
            self.action_shape = env.action_space.shape
            print("action range", self.action_low, self.action_high)
    
    def get_action(self, state):
        if self.is_discrete:
            action = random.choice(range(self.action_size))
        else:
            action = np.random.uniform(self.action_low, 
                                       self.action_high, 
                                       self.action_shape)
#         pole_angle = state[2]
#         action = 0 if pole_angle<0 else 1
        return action

In [14]:
class QNAgent(Agent):
    def __init__(self, env, discount_rate=0.97, learning_rate=0.01):
        super().__init__(env)
        self.state_size = env.observation_space.n
        print("state size", self.state_size)
        
        self.eps = 1.0
        self.discount_rate = discount_rate
        self.learning_rate = learning_rate
        self.build_model()
        
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        
    def build_model(self):
        tf.reset_default_graph()
        self.state_in = tf.placeholder(tf.int32, shape=[1])
        self.action_in = tf.placeholder(tf.int32, shape=[1])
        self.target_in = tf.placeholder(tf.float32, shape=[1])
        
        self.state = tf.one_hot(self.state_in, depth=self.state_size)
        self.action = tf.one_hot(self.action_in, depth=self.action_size)
        
        self.q_state = tf.layers.dense(self.state, units=self.action_size, name='q_table')
        self.q_action = tf.reduce_sum(tf.multiply(self.q_state, self.action), axis=1)
        
        self.loss = tf.reduce_sum(tf.square(self.target_in - self.q_action))
        self.optimizer = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss)
        
    def get_action(self, state):
        q_state = self.sess.run(self.q_state, feed_dict={self.state_in: [state]})
        action_greedy = np.argmax(q_state)
        action_random = super().get_action(state)        
        return action_random if random.random() < self.eps else action_greedy
    
    def train(self, experience):
        state, action, next_state, reward, done = [[exp] for exp in experience]
        
        q_next = self.sess.run(self.q_state, feed_dict={self.state_in: next_state})
        q_next[done] = np.zeros([self.action_size])
        q_target = reward + self.discount_rate * np.max(q_next)
        
        feed = {self.state_in: state, self.action_in: action, self.target_in: q_target}
        self.sess.run(self.optimizer, feed_dict=feed)
        
        if experience[4]:
            self.eps = self.eps * 0.99
            
    def __del__(self):
        self.sess.close()
        
agent = QNAgent(env)        

action size 4
state size 16


In [18]:
total_reward = 0
for ep in range(100):
    state = env.reset()
    done = False
    while not done:
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        agent.train((state, action, next_state, reward, done))
        state = next_state
        total_reward += reward
        
        print("s", state, "a", action)
        print("Episode: {}, Total Reward: {}, eps: {}".format(ep, total_reward, agent.eps))
        env.render()
        with tf.variable_scope('q_table', reuse=True):
            weights = agent.sess.run(tf.get_variable("kernel"))
            print(weights)
        time.sleep(0.05)
        clear_output(wait=True)
    
env.close()

s 15 a 2
Episode: 99, Total Reward: 98.0, eps: 0.017950553275045134
  (Right)
SFFF
FHFH
FFFH
HFFG
[[ 0.05454009  0.16960515  0.13788648  0.16167156]
 [ 0.13101447 -0.32750037  0.07599837 -0.04885117]
 [ 0.07776953  0.09806623 -0.04815603 -0.03669072]
 [ 0.06132403 -0.37249067 -0.02815397 -0.00752775]
 [ 0.04587616  0.19853249 -0.5580584   0.18744776]
 [ 0.03387719  0.26786274 -0.17599726 -0.45312062]
 [-0.5636495   0.26093155 -0.44426194 -0.02807396]
 [-0.49027243 -0.11480892 -0.00658172  0.21941489]
 [ 0.04223654 -0.58268374  0.3649092   0.23400971]
 [ 0.12436999  0.22099635  0.4008777  -0.6013224 ]
 [ 0.14905518  0.3015135  -0.5536058   0.27473304]
 [-0.3720506  -0.3678636  -0.4296068   0.30315918]
 [ 0.48901272  0.14052075  0.27998793  0.23519087]
 [-0.38213077  0.1019894   0.39910188 -0.02483442]
 [ 0.1722834   0.1770214   0.45701417  0.27047232]
 [-0.17119858  0.04632545 -0.22230217  0.08289915]]
